In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from sklearn.dummy import DummyClassifier
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PrefixTuningConfig, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from peft import PeftModel, PeftConfig
from glob import glob 
from collections import defaultdict, Counter
from os.path import basename


import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # replace the 0 with other gpu ids
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.set_device(0)
device = "cuda:0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so...


# Talkdown

In [2]:
td_dir = '/shared/2/projects/contextual-appropriateness/data/talkdown/'

talkdown_train_df = pd.read_json(td_dir + 'balanced_train.jsonl', lines=True)
talkdown_test_df = pd.read_json(td_dir + 'balanced_test.jsonl', lines=True)

print(len(talkdown_train_df))

#talkdown_train_df.drop_duplicates(inplace=True,ignore_index=True, subset='quotedpost', keep="first")
print(len(talkdown_train_df))
talkdown_train_df.head()

5208
5208


,quotedpost,quotedreply,label,post,reply,post_user,reply_user,start_offset,end_offset,reddit_post_id,reddit_reply_id
0,Please educate yoyrself before you bring your ...,"Not condescending at all, jeez.",True,"Well a guy is saying Barra, who has those grea...",> Please educate yoyrself before you bring you...,StalinHimself,Kel_Casus,135,208,dbl4vl9,dblfraj
1,There might be some small piece that's incorrect,You said that. Not me. Not James-Cizuz. You sa...,True,> I think you're the one who has a reading com...,> theories are constantly growing and evolving...,kishi,jids,365,413,c2dtpq9,c2dtywp
2,If I try and force down a breakfast I start ga...,Yes!\n\nPeople were so condescending about it ...,False,For me it's like temporarily having the flu. T...,> If I try and force down a breakfast I start ...,amphetaminesfailure,CowGiraffe,331,383,cuv97mf,cuvnb27
3,Nonviolence leads to reform and progress. It's...,"I don't believe you, you provide no examples, ...",True,"Uh, no. Nonviolence leads to reform and progre...",>Nonviolence leads to reform and progress. It'...,UmamiSalami,ArizonaIcedOutBoys,8,102,cmb6odf,cmbb0la
4,Have you actually read the bible ??,Is that where you learned how to be condescend...,True,>Do you not understand how insensitive and rud...,> Have you actually read the bible ??\n\nIs th...,lju1977,katapliktikos,58,93,cqq300a,cqq3289


In [3]:
relationship_to_verbalization = {
    "friend": "a person to their friend",
    "parent":"a parent to their child",
    "child":"a child to their parent",
    "sibling":"a person to their sibling",
    "best friend":"a person to their best friend",
    "best_friend":"a person to their best friend",
    "neighbor":"a person to their neighbor",
    "coworker":"a person to their coworker",
    "boss":"a boss to their employee",
    "direct report (employee)":"a person to their boss",
    "direct_report":"a person to their boss",    
    "student":"a student to their teacher",
    "teacher":"a teacher to their student",
    "cousins":"a person to their cousin",
    "granparent":"a person to their grandchild",
    "grandparent":"a person to their grandchild",
    "grandchild":"a person to their grandparent",    
    "uncle/aunt":"an uncle/aunt to their niece or nephew",
    "uncle_aunt":"an uncle or aunt to their niece or nephew",    
    "neice_nephew":"a niece or nephew to their uncle or aunt",        
    "employee_in_large_company":"an employee in large company to another",
    "married":"a person to their spouse",
    "dating":"a person to the person they are dating",
    "engaged":"a person to their fiancee",
    "friends_with_benefits":"a person to their friend with benefits",
    "divorcee":"a person to their ex-spouse",
    "ex-girlfriend/ex-boyfriend":"a person to their ex-girlfriend or ex-boyfriend",
    "ex_dating":"a person to their ex-girlfriend or ex-boyfriend",    
    "step-sibling":"a person to their step-sibling",
    "step_sibling":"a person to their step-sibling",    
    "fan":"a fan to their hero",
    "hero":"a hero to their fan",
    "enemy":"a person to their enemy",
    "rival":"a person to their rival",
    "competitor":"a person to their competitor",
    "complete_stranger":"a person to a complete stranger",
    "acquaintance":"a person to an acquaintance",
    "person_with_authority":"a person with authority to a subordinate",
    "law_enforcement":"a member of law enforcement to a community member",
    "classmate":"a person to their classmate",
    "sports_teammate":"a person to their sports teammate",
    "club_member":"a person to a member of their club",
    "adopted_child":"an adopted child to their parent",
    "adopted child":"an adopted child to their parent",
    "domestic_partner":"a person to their domestic partner",
    "person_having_an_affair":"a person having an affair to their partner",
    "mentor":"a mentor to their mentee",
    "mentee":"a mentee to their mentor",
    "landlord":"a landlord to their tenant",
    "colleague":"a person to their colleague",
    "childhood_friend":"a person to their childhood friend",
    "old_friend":"a person to an old friend",
    "client":"a client to their lawyer",
    "lawyer":"a lawyer to their client",
    "patient":"a patient to their doctor",
    "doctor":"a doctor to their patient",
    #"lover":"a person to their lover", #relationships in PRIDE
    #"commercial":"a person to someone in a commercial relationship",#relationships in PRIDE
    #"church":"a person to someone in their church" #relationships in PRIDE    
}
len(relationship_to_verbalization)

56

In [4]:
datadir = '/shared/2/projects/contextual-appropriateness/data/final-annotated-data-cleaned/'
train_df = pd.read_csv(datadir + 'train.csv')
rels = set(train_df.relationship)
len(rels)

49

In [5]:
rel_to_tuple = {}
for rel, verbalization in relationship_to_verbalization.items():
    #print(verbalization)
    i = verbalization.index('to ')
    v1 = verbalization[:i-1]
    v2 = verbalization[i:].replace('to their ', '').replace('to ', '')
    rel_to_tuple[rel] = (v1, v2)

In [7]:
#rel_to_tuple

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_name_or_path = "google/flan-t5-xl"
tokenizer_name_or_path = "google/flan-t5-xl"
best_model_dir = '/shared/2/projects/contextual-appropriateness/models/peft/google/flan-t5-xl/seed-12345//best'

text_column = "text1"
label_column = "answer"
max_length = 196
lr = 1e-2
num_epochs = 20

In [9]:
def ntok(text):
    return len(tokenizer(text)['input_ids'])

def shrink(quote, reply, total_prompt_len):

    qids = tokenizer(quote)['input_ids']
    rids = tokenizer(reply)['input_ids']   
    plen = total_prompt_len - (len(qids) + len(rids))
    
    surplus = len(qids) + len(rids) - (max_length - plen)
    
    # Try trimming the quote first
    if len(qids) > 50:               
        qtrim = max(50, len(qids) - surplus)
        qids = qids[:qtrim]
        
    surplus = len(qids) + len(rids) - (max_length - plen)


    qtrim = max(10, len(qids)-int(surplus/2))    
    qids = qids[:qtrim]
    surplus = len(qids) + len(rids) - (max_length - plen)
    rids = rids[:len(rids)-surplus]
            
    quote = tokenizer.decode(qids)
    reply = tokenizer.decode(rids)
            
    return quote, reply       

def verbalize(rel, quote, reply):
    parties = rel_to_tuple[rel]
    rel_phrase = relationship_to_verbalization[rel]
        
    text1 = ('Rate whether it is inappropriate for message A to be said in response ' + \
            'to the message B in the specified social setting:\n' + \
            'A: %s\n' + \
            'B: %s\n' + \
            'setting: %s\n' + \
            'answer (yes or no):') % (reply, quote, rel_phrase)

    l = ntok(text1)
    if l > max_length:
        quote, reply = shrink(quote, reply, l)
        text1 = ('Rate whether it is inappropriate for message A to be said in response ' + \
                'to the message B in the specified social setting:\n' + \
                'A: %s\n' + \
                'B: %s\n' + \
                'setting: %s\n' + \
                'answer (yes or no):') % (reply, quote, rel_phrase)            
        
    text2 = 'Is inappropriate for a %s to say "%s" to a %s in response to %s, yes or no?' \
        % (parties[0], reply, parties[1], quote)
            
    #return pd.Series({'format1': text1, 'format2': text2})
    return text1, text2

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [11]:
tokenizer.is_fast

True

In [12]:
all_rel_talkdown_df = defaultdict(list)
for row in tqdm(talkdown_train_df.itertuples(), total=len(talkdown_train_df)):
    for rel in rels:
        all_rel_talkdown_df['reddit_reply_id'].append(row.reddit_reply_id)
        t1, t2 = verbalize(rel, row.quotedpost, row.quotedreply)
        all_rel_talkdown_df['text1'].append(t1)
        all_rel_talkdown_df['text2'].append(t2)       
        all_rel_talkdown_df['relationship'].append(rel)
        all_rel_talkdown_df['label'].append(row.label)
        
all_rel_talkdown_df = pd.DataFrame(all_rel_talkdown_df)
all_rel_talkdown_df['answer'] = 'yes'
len(all_rel_talkdown_df)

  0%|          | 0/5208 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (751 > 512). Running this sequence through the model will result in indexing errors


255192

In [14]:
all_rel_talkdown_test_df = defaultdict(list)
for row in tqdm(talkdown_test_df.itertuples(), total=len(talkdown_test_df)):
    for rel in rels:
        all_rel_talkdown_test_df['reddit_reply_id'].append(row.reddit_reply_id)
        t1, t2 = verbalize(rel, row.quotedpost, row.quotedreply)
        all_rel_talkdown_test_df['text1'].append(t1)
        all_rel_talkdown_test_df['text2'].append(t1)       
        all_rel_talkdown_test_df['relationship'].append(rel)
        all_rel_talkdown_test_df['label'].append(row.label)
        
all_rel_talkdown_test_df = pd.DataFrame(all_rel_talkdown_test_df)
all_rel_talkdown_test_df['answer'] = 'yes'
len(all_rel_talkdown_test_df)

  0%|          | 0/652 [00:00<?, ?it/s]

31948

## Save the dataframes because they're expensive to compute

In [15]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_talkdown_df.to_csv(peft_outdir + 'talkdown.balanced.train.csv', index=False)
all_rel_talkdown_test_df.to_csv(peft_outdir + 'talkdown.balanced.test.csv', index=False)

In [5]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_talkdown_df = pd.read_csv(peft_outdir + 'talkdown.balanced.train.csv')
all_rel_talkdown_test_df = pd.read_csv(peft_outdir + 'talkdown.balanced.test.csv')

In [17]:
all_rel_talkdown_train_ds = Dataset.from_pandas(all_rel_talkdown_df)
all_rel_talkdown_test_ds = Dataset.from_pandas(all_rel_talkdown_test_df)
all_rel_talkdown_dd = DatasetDict()
all_rel_talkdown_dd['train'] = all_rel_talkdown_train_ds
all_rel_talkdown_dd['test'] = all_rel_talkdown_test_ds

In [18]:
def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [19]:
processed_all_rel_talkdown = all_rel_talkdown_dd.map(
    preprocess_function,
    batched=True,
    num_proc=32,
    remove_columns=all_rel_talkdown_dd["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset (num_proc=32):   0%|          | 0/127596 [00:00<?, ? examples/s]

Running tokenizer on dataset (num_proc=32):   0%|          | 0/15974 [00:00<?, ? examples/s]

In [20]:
config = PeftConfig.from_pretrained(best_model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, best_model_dir)
model.to(device)
model.eval()
print('model loaded')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded


In [21]:
batch_size = 154

train_dataloader = DataLoader(
    processed_all_rel_talkdown['train'], shuffle=False, collate_fn=default_data_collator, 
    batch_size=batch_size, pin_memory=True
)

train_preds = []
for step, batch in enumerate(tqdm(train_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    train_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )  

  0%|          | 0/829 [00:00<?, ?it/s]

In [25]:
batch_size = 154

test_dataloader = DataLoader(
    processed_all_rel_talkdown['test'], shuffle=False, collate_fn=default_data_collator, 
    batch_size=batch_size, pin_memory=True
)

test_preds = []
for step, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    test_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )  

  0%|          | 0/104 [00:00<?, ?it/s]

In [26]:
all_rel_talkdown_df['predicted'] = train_preds
all_rel_talkdown_test_df['predicted'] = test_preds

<ipython-input-26-17eef78e7883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part1['predicted'] = train_preds
<ipython-input-26-17eef78e7883>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_part1['predicted'] = test_preds


In [28]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_talkdown_df.to_csv(peft_outdir + 'talkdown.train.balanced.labeled.csv', index=False)
all_rel_talkdown_test_df.to_csv(peft_outdir + 'talkdown.test.balanced.labeled.csv', index=False)